In [1]:
import sys
import pandas as pd
import numpy as np
import spotipy 
sp=spotipy.Spotify()

In [2]:
from bokeh.embed import file_html

In [3]:
## Get information on each of the albums for a given artist 
def fetch_albums(musician): 
    artist=sp.search(q='artist:'+musician, type='artist')
    if (len(artist['artists'])==0): 
        print 'Error: Check Musician Name'
        return None
    else: 
        ## Pull album information 
        albums=sp.artist_albums(artist['artists']['items'][0]['id'], album_type='album', country='US')
        df_album=pd.DataFrame()
        for i in range(len(albums['items'])): 
            df_album.loc[i,'name']=albums['items'][i]['name']
            df_album.loc[i,'id']=albums['items'][i]['id']
        for j in range(len(df_album)): 
            album_info=sp.album(df_album.loc[j,'id'])
            df_album.loc[j,'release_date']=album_info['release_date']
            df_album.loc[j,'popularity']=album_info['popularity']
            
    return df_album


In [4]:
def fetch_tracks(df_album): 
    tracks=pd.DataFrame()
    print 'Querying Album Tracks'
    for i in range(len(df_album)):
        test=sp.album_tracks(df_album.loc[i,'id'])
        for j in range(len(test['items'])): 
            k=len(tracks)
            tracks.loc[k,'album']=df_album.loc[i,'name']
            tracks.loc[k, 'name']=test['items'][j]['name']
            tracks.loc[k,'track_id']=test['items'][j]['id']
            tracks.loc[k,'artist_num']=len(test['items'][j]['artists'])

    for i in range(len(tracks)): 
        if (i%20==0): 
            print i 
        test=sp.track(tracks.loc[i,'track_id'])
        tracks.loc[i,'popularity']=test['popularity']
        tracks.loc[i,'explicit']=test['explicit']
        if (tracks.loc[i, 'artist_num']>1): 
            stringer=''
            for j in range(0,tracks.loc[i, 'artist_num'].astype(int)): 
                if (j<tracks.loc[i, 'artist_num']-1):
                    namer=test['artists'][j]['name']
                    stringer=stringer+test['artists'][j]['name']+', '
                else: 
                    stringer=stringer+test['artists'][j]['name']
            tracks.loc[i,'artists_on_track']=stringer
            
    return tracks 


In [29]:
# Create list of collaborators
import unicodedata

def make_collab_list(df_tracks, main_artist):
    collaborators_list=[]
    for i in range(len(df_tracks)): 
        if (df_tracks.loc[i,'artist_num']>1): 
            test1=unicodedata.normalize('NFKD', df_tracks.loc[i,'artists_on_track']).encode('ascii','ignore')
            test=test1.split(',')
            for j in range(len(test)): 
                if (test[j].lstrip()!=main_artist) : 
                    name=test[j].lstrip()
                    matching = [s for s in collaborators_list if name in s]
                    if (len(matching)==0):
                        collaborators_list.append(name)
                        
        collab=pd.DataFrame()
        collab['collaborator']=collaborators_list

    for i in range(len(collab)): 
        person=collab.loc[i,'collaborator']
        ### BETTER WAY TO SORT OUT UNICODE ISSUES? 
        if (person=='Beyonce'): 
            person='Beyo'
        temp=df_tracks.loc[np.where((df_tracks['artists_on_track'].str.contains(person))==True)[0],['album', 'artists_on_track','popularity']]
        collab.loc[i, 'num_collab']=len(temp)
        collab.loc[i, 'mean_pop']=np.mean(temp['popularity'])
        collab.loc[i, 'std_pop']=np.std(temp['popularity'])
    
    return collab

In [5]:
kw_albums=fetch_albums('Kanye West')

In [6]:
kw_albums

,name,id,release_date,popularity
0,The Life Of Pablo,0WAuEfa5Lmg72xfydLVcca,2016-04-04,95
1,Yeezus,7D2NdGvBHIavgLhmcwhluK,2013-06-18,78
2,Watch The Throne (Explicit Version),7mCeLbChyegbRwwKK5shJs,2011-08-12,71
3,Watch The Throne (Edited Version),0eZRIt3Ht1IuMXsr0QR8OY,2011-08-12,42
4,Watch The Throne [Deluxe Edition (Explicit)],1YwzJz7CrV9fd9Qeb6oo1d,2011-08-12,80
5,Watch The Throne [Deluxe Edition (Edited)],4P63UgNDUcF11MnWzyvVrh,2011-08-12,51
6,My Beautiful Dark Twisted Fantasy (Explicit Ve...,20r762YmB5HeofjMCiPMLv,2010-11-19,81
7,My Beautiful Dark Twisted Fantasy (Edited Vers...,6klUp8sQyRXGuJhqZu4PG3,2010-11-22,51
8,808s & Heartbreak,2JK89jt4unItFroOr0kT3g,2008-11-24,75
9,Graduation (Explicit Version),3SZr5Pco2oqKFORCP3WNj9,2007-09-11,81


In [7]:
tracks=fetch_tracks(kw_albums)

Querying Album Tracks
0
20
40
60
80
100
120
140
160
180
200
220


There are multiple versions of individual albums (explicit/clean, standard/deluxe). First let's see if whether or not the track is explicit affects its popularity

In [8]:
ex_hist, ex_edges = np.histogram(tracks.loc[np.where(tracks['explicit']==True)[0],'popularity'],range=[0,100], bins=25)
ex_center=(ex_edges[:-1]+ex_edges[1:])/2.

cl_hist, cl_edges = np.histogram(tracks.loc[np.where(tracks['explicit']==False)[0],'popularity'],range=[0,100], bins=25)
cl_center=(cl_edges[:-1]+cl_edges[1:])/2.

histo=pd.DataFrame()
histo['explicit']=ex_hist
histo['ex_center']=ex_center
histo['clean']=cl_hist
histo['cl_center']=cl_center

In [9]:
from bokeh.plotting import figure, show, vplot
from bokeh.charts import Histogram,Bar, output_file, show, output_notebook
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend

p = figure(width=600, height=400)

p = Bar(histo, label='ex_center', 
        values=blend('explicit', 'clean', labels_name='typer'),  
        stack=cat(columns='typer', sort=False),
        color=color(columns='typer', palette=['red', 'blue'],sort=False), 
        legend='top_right', xlabel='Popularity', ylabel='Number',
        tooltips=[('number', '@typer'), ('popularity', '@ex_center')])


print 'Mean Popularity (Explicit): ', np.mean(tracks.loc[np.where(tracks['explicit']==True)[0],'popularity']),'+-',np.std(tracks.loc[np.where(tracks['explicit']==True)[0],'popularity'])
print 'Mean Popularity (Clean): ', np.mean(tracks.loc[np.where(tracks['explicit']==False)[0],'popularity']),'+-',np.std(tracks.loc[np.where(tracks['explicit']==False)[0],'popularity'])

output_notebook()

#show(p)

Mean Popularity (Explicit):  58.9596774194 +- 12.2332170706
Mean Popularity (Clean):  31.871559633 +- 12.1656011646


Loading BokehJS ...

In [15]:
from bokeh.plotting import figure, show, vplot
from bokeh.charts import Histogram,Bar, output_file, show, output_notebook
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool    

p = figure(width=600, height=400)

p.quad(top=ex_hist, bottom=0, left=ex_edges[:-1],right=ex_edges[1:],\
       fill_color="white",line_color="red", legend="Explicit")
p.quad(top=cl_hist, bottom=0, left=cl_edges[:-1], right=cl_edges[1:],\
      fill_color="white", line_color="blue", fill_alpha=0.2, legend="Clean")

p.legend.location="top_right"
p.xaxis.axis_label="Popularity"
p.yaxis.axis_label="Number of Tracks"

print 'Mean Popularity (Explicit): ', np.mean(tracks.loc[np.where(tracks['explicit']==True)[0],'popularity']),'+-',np.std(tracks.loc[np.where(tracks['explicit']==True)[0],'popularity'])
print 'Mean Popularity (Clean): ', np.mean(tracks.loc[np.where(tracks['explicit']==False)[0],'popularity']),'+-',np.std(tracks.loc[np.where(tracks['explicit']==False)[0],'popularity'])

output_notebook()

#show(p)

Mean Popularity (Explicit):  58.9596774194 +- 12.2332170706
Mean Popularity (Clean):  31.871559633 +- 12.1656011646


Loading BokehJS ...

In [12]:
from bokeh.embed import components
script, div = components(p)

print div

f=open('p1.js', 'w')
f.write(script)
f.close()


<div class="plotdiv" id="89556313-b589-45bc-baaf-a3e933260621"></div>


Overall, the explicit version of Kanye songs are significantly more popular than the clean versions. I suppose that makes 
sense in a go big or go home sort of way. If you're going to listen to Kanye, listen to KANYE. 

However the distributions show that some clean versions of songs are as popular as other explicit versions. Let's look at how the difference in popularity between song versions varies by song. 

In [13]:
## Need to clean up song titles such that I can group them cleanly in pandas
pd.set_option('display.max_rows', 200)
indexer=np.where(tracks['name'].str.contains('(Edited)')==True)[0]
tracks['ed_name']=tracks['name']
tracks.loc[indexer, 'ed_name']=tracks.loc[indexer, 'name'].str[:-24]
tracks['ed_name']=tracks['ed_name'].str.rstrip()


/Users/kschles/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/strings.py:189: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)


In [14]:
tracks_temp=pd.DataFrame()
tracks_temp=tracks
tracks_temp['pop_max']=tracks['popularity']
tracks_temp['pop_diff']=tracks['popularity']

grouped=tracks_temp[['ed_name', 'album','popularity', 'pop_max', 'pop_diff']].groupby('ed_name')

pop_diff=grouped.agg({'ed_name' : 'first', 'album': 'first', 'popularity' : 'first', 'pop_max' : 'max', 'pop_diff' : lambda x: np.max(x)-np.min(x)})
pop_diff.reset_index(drop=True, inplace=True)

In [16]:
# Remove songs where there is only one version 
pop_diff_ed=pop_diff.loc[np.where(pop_diff['pop_diff']!=0)[0]]

In [22]:
from bokeh.charts import Scatter, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool    
from scipy import stats 

source = ColumnDataSource(
        data=dict(
            x=pop_diff_ed['pop_max'].astype(int),
            y=pop_diff_ed['pop_diff'].astype(int),
            album=pop_diff_ed['album'].astype(str),
            song=pop_diff_ed['ed_name'].astype(object),
            per=pop_diff_ed['pop_diff']/pop_diff_ed['pop_max'],
        )
    )

TOOLS = [BoxZoomTool(), ResetTool(),HoverTool(tooltips=[("song", "@song"),("album", "@album")])]

p2 = figure(plot_width=600, plot_height=400, tools=TOOLS)
p2.xaxis.axis_label = "Popularity (Explicit Version)"
p2.yaxis.axis_label = "Difference in Popularity"

colormap = {'Graduation (Explicit Version)': 'red', 'Late Registration (Explicit Version)': 'green',\
            'My Beautiful Dark Twisted Fantasy (Explicit Version)': 'blue', \
           'The College Dropout (Explicit)': 'purple', \
            'Watch The Throne (Explicit Version)': 'cyan',\
            'Watch The Throne [Deluxe Edition (Explicit)]' : 'black'}
pop_diff_ed['color'] = pop_diff_ed['album'].map(lambda x: colormap[x])

slope, intercept, r_value, p_value, std_err = stats.linregress(source.data['x'],source.data['y'])
xvalue=np.arange(0, 100, dtype=np.float)
yvalue=slope*xvalue+intercept

p2.line(xvalue, yvalue, line_dash="4 4", line_width=2, color='black')


p2.circle('x', 'y', size=7, color=pop_diff_ed['color'], \
         fill_alpha=0.2, source=source)

output_notebook()

#show(p2)


Loading BokehJS ...

In [21]:
from bokeh.embed import components
script, div = components(p2)

print div

f=open('p2.js', 'w')
f.write(script)
f.close()


<div class="plotdiv" id="4095a633-eecd-4c3f-bc5c-cf33f2d49846"></div>


In general, the differences between the popularity of the explicit and clean version increases as songs get more popular. I suppose this makes sense, as the popularity index takes into account number of plays. 

The biggest discrepancy is for the most popular song, 'Ni**as in Paris.' And for that song, I can't even imagine the clean version!

For the rest of this analysis, I'm going to just use the explicit versions of tracks when available. There also is a Deluxe version of one album, 'Watch the Throne'. I'll remove that one as well for now, and just stick with the regular explicit version. 

In [23]:
explicit=tracks.loc[np.where((tracks['explicit']==True) & (tracks['album'].str.contains('Deluxe')==False))[0]]
explicit.reset_index(drop=True,inplace=True)

Now that I've trimmed all the tracks down to the explicit versions, I want to see how Kanye alone fares against Kanye and Friends. Let's see how the distribution of popularity compares for these two categories. 

In [24]:
ex_hist, ex_edges = np.histogram(explicit.loc[np.where(explicit['artist_num']==1)[0],'popularity'],range=[0,100], bins=25)
ex_center=(ex_edges[:-1]+ex_edges[1:])/2.
## Fractional: 
#ex_hist=(ex_hist.astype(float))/float(sum(ex_hist))

cl_hist, cl_edges = np.histogram(explicit.loc[np.where(explicit['artist_num']>1)[0],'popularity'],range=[0,100], bins=25)
cl_center=(cl_edges[:-1]+cl_edges[1:])/2.
## Fractional: 
#cl_hist=(cl_hist.astype(float))/float(sum(cl_hist))

histo=pd.DataFrame()
histo['alone']=ex_hist
histo['ex_center']=ex_center
histo['collaboration']=cl_hist
histo['cl_center']=cl_center


In [31]:
from bokeh.plotting import figure, show, vplot
from bokeh.charts import Histogram,Bar, output_file, show, output_notebook
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend

p = figure(width=300, height=300)

p = Bar(histo, label='ex_center', 
        values=blend('alone', 'collaboration', labels_name='typer'),  
        stack=cat(columns='typer', sort=False),
        color=color(columns='typer', palette=['red', 'blue'], sort=False), 
        legend='top_right', xlabel='Popularity', ylabel='Number',
        tooltips=[('number', '@typer'), ('popularity', '@ex_center')])

print 'Average Track Popularity (Kanye Alone): ', np.mean(explicit.loc[np.where(explicit['artist_num']==1)[0],'popularity']), '+-', np.std(tracks.loc[np.where(tracks['artist_num']==1)[0],'popularity'])
print 'Average Track Popularity (with Collaborators): ', np.mean(explicit.loc[np.where(explicit['artist_num']!=1)[0],'popularity']), '+-', np.std(tracks.loc[np.where(tracks['artist_num']!=1)[0],'popularity'])

output_notebook()
#show(p)

Average Track Popularity (Kanye Alone):  61.1818181818 +- 19.8423146715
Average Track Popularity (with Collaborators):  56.5476190476 +- 15.8097893281


Loading BokehJS ...

In [27]:
from bokeh.plotting import figure, show, vplot
from bokeh.charts import Histogram,Bar, output_file, show, output_notebook
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool    

p = figure(width=600, height=400)

p.quad(top=ex_hist, bottom=0, left=ex_edges[:-1],right=ex_edges[1:],\
       fill_color="white",line_color="red", legend="Solo")
p.quad(top=cl_hist, bottom=0, left=cl_edges[:-1], right=cl_edges[1:],\
      fill_color="white", line_color="blue", fill_alpha=0.2, legend="Collaboration")

p.legend.location="top_right"
p.xaxis.axis_label="Popularity"
p.yaxis.axis_label="Number of Tracks"

print 'Average Track Popularity (Kanye Alone): ', np.mean(explicit.loc[np.where(explicit['artist_num']==1)[0],'popularity']), '+-', np.std(tracks.loc[np.where(tracks['artist_num']==1)[0],'popularity'])
print 'Average Track Popularity (with Collaborators): ', np.mean(explicit.loc[np.where(explicit['artist_num']!=1)[0],'popularity']), '+-', np.std(tracks.loc[np.where(tracks['artist_num']!=1)[0],'popularity'])

output_notebook()

#show(p)

Average Track Popularity (Kanye Alone):  61.196969697 +- 19.9641742071
Average Track Popularity (with Collaborators):  56.5 +- 15.7944300594


Loading BokehJS ...

In [26]:
from bokeh.embed import components
script, div = components(p)

print div

f=open('p3.js', 'w')
f.write(script)
f.close()


<div class="plotdiv" id="a4fa9f83-d641-441f-8c4d-c8624105d740"></div>


In general, the distributions of popularity for Kanye with and without collaborators are pretty similar, although Kanye alone is slightly more popular. 

Let's see which other musicians have the largest affect on song popularity... 

In [30]:
collab_info=make_collab_list(explicit, 'Kanye West')

In [31]:
collab_info['count']=pd.Series(range(0, len(collab_info)))

Now, I'm going to look at the mean popularity of songs for each collaborator and compare it to the popularity of Kanye alone, e.g., who drags him down and who raises him up. 

In [43]:
explicit.loc[np.where(explicit['name']=='Homecoming')[0]]

,album,name,track_id,artist_num,popularity,explicit,artists_on_track,ed_name,pop_max,pop_diff
65,Graduation (Explicit Version),Homecoming,2iaCM7WvOknQI1230hA9eK,2,14,True,"Kanye West, Chris Martin",Homecoming,14,14


In [48]:
from bokeh.charts import Scatter, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool    

source = ColumnDataSource(
        data=dict(
            x=collab_info['count'],
            y=collab_info['mean_pop'].astype(int),
            person=collab_info['collaborator'].astype(object),
        )
    )

TOOLS = [BoxZoomTool(), ResetTool(), HoverTool(tooltips=[("Collaborator","@person")])]

kanye_mean=np.mean(explicit.loc[np.where(explicit['artist_num']==1)[0],'popularity'])
kanye_std=np.std(explicit.loc[np.where(explicit['artist_num']==1)[0],'popularity'])

collab_mean=np.mean(explicit.loc[np.where(explicit['artist_num']>1)[0],'popularity'])
collab_std=np.std(explicit.loc[np.where(explicit['artist_num']>1)[0],'popularity'])

collab_mean2=np.mean(explicit.loc[np.where((explicit['artist_num']>1) & (explicit['name']!='Homecoming'))[0],'popularity'])
collab_std2=np.std(explicit.loc[np.where(explicit['artist_num']>1)[0],'popularity'])


p = figure(plot_width=600, plot_height=500, tools=TOOLS)
p.xaxis.axis_label = "Collaborator"
p.yaxis.axis_label = "Mean Popularity of Collaboration"
p.circle(collab_info.loc[np.where(collab_info['collaborator']=='JAY Z')[0],'count'], collab_info.loc[np.where(collab_info['collaborator']=='JAY Z')[0],'mean_pop'], size=12, color='gold')
p.circle('x', 'y', size=7, color='black', source=source)
p.xaxis.major_label_text_font_size = '0pt'

p.line([-1,45],[kanye_mean, kanye_mean], line_width=2, color='red')
p.line([-1,45],[(kanye_mean+kanye_std), (kanye_mean+kanye_std)], line_dash="4 4", line_width=2, color='red')
p.line([-1,45],[(kanye_mean-kanye_std), (kanye_mean-kanye_std)], line_dash="4 4", line_width=2, color='red')

p.line([-1,45],[collab_mean, collab_mean], line_width=2, color='blue')
#p.line([-1,45],[collab_mean2, collab_mean2], line_width=2, color='blue', line_dash="4 4")
#p.line([-1,45],[(collab_mean+collab_std), (collab_mean+collab_std)], line_dash="4 4", line_width=2, color='blue')
#p.line([-1,45],[(collab_mean-collab_std), (collab_mean-collab_std)], line_dash="4 4", line_width=2, color='blue')

#show(p)

In [47]:
from bokeh.embed import components
script, div = components(p)

print div

f=open('p4.js', 'w')
f.write(script)
f.close()


<div class="plotdiv" id="29cc6972-fa67-4afc-91d4-ab9049b59e20"></div>


Most of the collaborative tracks are well within the range of popularity for Kanye alone. Except for that one little guy. Which leads to the all important question: do Kanye fans hate Chris Martin?!

The song 'Homecoming' is from the album 'Graduation.' 

In [49]:
explicit_albums=kw_albums.loc[np.where((kw_albums['name'].str.contains('Deluxe')==False) & (kw_albums['name'].str.contains('Edited')==False))[0]]
explicit_albums.reset_index(inplace=True, drop=True)
explicit_albums['count']=pd.Series(range(0, len(explicit_albums)))

In [50]:
explicit_albums['year']=explicit_albums['release_date'].astype(str).str[0:4].astype(int)

In [58]:
from bokeh.charts import Scatter, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool    

source = ColumnDataSource(
        data=dict(
            x=explicit_albums['year'],
            y=explicit_albums['popularity'],
            album=explicit_albums['name'].astype(object),
        )
    )

TOOLS = [BoxZoomTool(), ResetTool(), HoverTool(tooltips=[("album","@album")])]

meaner=np.mean(explicit_albums['popularity'])
stder=np.std(explicit_albums['popularity'])

p = figure(plot_width=600, plot_height=500, tools=TOOLS)
p.xaxis.axis_label = "Album Year"
p.yaxis.axis_label = "Popularity"
p.circle(explicit_albums.loc[5,'year'], explicit_albums.loc[5,'popularity'], size=15, color='cyan')
p.circle('x', 'y', size=10, color='black', source=source)
#p.xaxis.major_label_text_font_size = '0pt'

p.line([2003,2017],[meaner, meaner], line_width=2, color='red')
p.line([2003,2017],[(meaner+stder), (meaner+stder)], line_dash="4 4", line_width=2, color='red')
p.line([2003,2017],[(meaner-stder), (meaner-stder)], line_dash="4 4", line_width=2, color='red')


#show(p)

In [57]:
from bokeh.embed import components
script, div = components(p)

print div

f=open('p5.js', 'w')
f.write(script)
f.close()


<div class="plotdiv" id="1e1f78b0-d703-46c3-b68d-5bff70b8ba96"></div>


Here I have marked 'Graduation' with a cyan border. As you can see , the album itself is above average in popularity. Additionally, the average is a probably skewed a bit high, as 'Life of Pablo' is a fairly new release, so is probably quite popular at the moment. 

In [217]:
from bokeh.charts import Scatter, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool    

source = ColumnDataSource(
        data=dict(
            x=explicit_albums['year'],
            y=explicit_albums['popularity'],
            album=explicit_albums['name'].astype(object),
        )
    )

TOOLS = [BoxZoomTool(), ResetTool(), HoverTool(tooltips=[("album","@album")])]

meaner=np.mean(explicit_albums['popularity'])
stder=np.std(explicit_albums['popularity'])

p = figure(plot_width=600, plot_height=500, tools=TOOLS)
p.xaxis.axis_label = "Release Year"
p.yaxis.axis_label = "Popularity"
p.circle(explicit_albums.loc[5,'year'], explicit_albums.loc[5,'popularity'], size=15, color='cyan')
p.circle('x', 'y', size=10, color='black', source=source)
p.xaxis.major_label_text_font_size = '0pt'

p.line([2000,2017],[meaner, meaner], line_width=2, color='red')
p.line([2000,2017],[(meaner+stder), (meaner+stder)], line_dash="4 4", line_width=2, color='red')
p.line([2000,2017],[(meaner-stder), (meaner-stder)], line_dash="4 4", line_width=2, color='red')


#show(p)

There is not a clear correlation between age of album and popularity. But 'Life of Pablo' is so recent, I'm guessing it's value is a bit biased. I mean it's good, but I'm not sure it's 2sigma good. 

Anyways, you can see that 'Graduation' as an album is pretty popular. So what's up with that Chris Martin collaboration?!

In [216]:
### Want to see how it did in the RIAA rankings. 

,name,id,release_date,popularity,count,year
0,The Life Of Pablo,0WAuEfa5Lmg72xfydLVcca,2016-04-04,95,0,2016
1,Yeezus,7D2NdGvBHIavgLhmcwhluK,2013-06-18,78,1,2013
2,Watch The Throne (Explicit Version),7mCeLbChyegbRwwKK5shJs,2011-08-12,70,2,2011
3,My Beautiful Dark Twisted Fantasy (Explicit Ve...,20r762YmB5HeofjMCiPMLv,2010-11-19,81,3,2010
4,808s & Heartbreak,2JK89jt4unItFroOr0kT3g,2008-11-24,75,4,2008
5,Graduation (Explicit Version),3SZr5Pco2oqKFORCP3WNj9,2007-09-11,81,5,2007
6,Late Registration (Explicit Version),5ll74bqtkcXlKE7wwkMq4g,2005-09-30,79,6,2005
7,The College Dropout (Explicit),3ff2p3LnR6V7m6BinwhNaQ,2004-02-13,77,7,2004


In [270]:
from bs4 import BeautifulSoup
import urllib2
wiki = "https://en.wikipedia.org/wiki/Kanye_West_discography#Singles"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [296]:
# The list of singles is the 5th table on the wikipage
singles = soup.findAll("table", { "class" : "wikitable plainrowheaders" })[4]

In [542]:
wiki_data=pd.DataFrame()

rows=singles.findAll("tr")
#Null value for billboard ranking 
nuller=(rows[12].findAll('td')[0])

for i in range(2,len(rows)-1): 
    row=rows[i]
    ## pull out song title
    wiki_data.loc[i,'title']=row.findAll("th")[0].findAll("a")[0].get('title')
    cells=row.findAll("td")
    ## pull out US chart ranking 
    """
    if ((cells[0]!=nuller)): 
        if ((cells[0].renderContents()>2000) & (cells[1]!=nuller)): 
            wiki_data.loc[i, 'us_rank']=int(cells[1].renderContents())
        else if (): 
            wiki_data.loc[i, 'us_rank']=int(cells[0].renderContents())

    ## pull out RIAA Rating
    #if (len(cells)>11): 
    #    if (cells[11].li!=None): 
            temp=(unicode(cells).strip())
            if (temp.find(u'RIAA: Gold')>-1): 
                wiki_data.loc[i,'riaa']='G'
            if (temp.find(u'RIAA: Platinum')>-1): 
                wiki_data.loc[i,'riaa']='P'
            if (temp.find(u'RIAA: 2x Platinum')>-1): 
                wiki_data.loc[i,'riaa']='2P'
            if (temp.find(u'RIAA: 3x Platinum')>-1): 
                wiki_data.loc[i,'riaa']='3P'
            if (temp.find(u'RIAA: 4x Platinum')>-1): 
                wiki_data.loc[i,'riaa']='4P'
            if (temp.find(u'RIAA: 5x Platinum')>-1): 
                wiki_data.loc[i,'riaa']='5P'
            if (temp.find(u'RIAA: 6x Platinum')>-1): 
                wiki_data.loc[i,'riaa']='6P'
            if (temp.find(u'RIAA: 7x Platinum')>-1): 
                wiki_data.loc[i,'riaa']='7P'
    """


In [497]:
"""
print i
row=rows[i]
cells=(row.findAll('td'))
print cells[1]
if (cells!=test) : 
    barf=cells[0]
    print 'Barf', barf
"""

"\nprint i\nrow=rows[i]\ncells=(row.findAll('td'))\nprint cells[1]\nif (cells!=test) : \n    barf=cells[0]\n    print 'Barf', barf\n"

In [543]:
## Giving up at the moment 
[2, ], ['G', '']

,title
2,Through the Wire
3,Slow Jamz
4,All Falls Down
5,Jesus Walks (Kanye West song)
6,The New Workout Plan
7,Diamonds from Sierra Leone
8,Gold Digger (Kanye West song)
9,Heard 'Em Say
10,Touch the Sky (Kanye West song)
11,Twista


I was trying to webscrape wikipedia to see if I can compare songs on popularity and RIAA rating. Homecoming is certified Platinum. 

In [546]:
explicit['count']=pd.Series(range(0, len(explicit)))

In [551]:
from bokeh.charts import Scatter, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool    

source = ColumnDataSource(
        data=dict(
            x=explicit['count'],
            y=explicit['popularity'],
            track=explicit['name'].astype(object),
        )
    )

TOOLS = [BoxZoomTool(), ResetTool(), HoverTool(tooltips=[("Track","@track")])]

meaner=np.mean(explicit['popularity'])
stder=np.std(explicit['popularity'])

p = figure(plot_width=600, plot_height=500, tools=TOOLS)
p.xaxis.axis_label = "track"
p.yaxis.axis_label = "Popularity"
p.circle(explicit.loc[np.where(explicit['name']=='Homecoming')[0],'count'], explicit.loc[np.where(explicit['name']=='Homecoming')[0],'popularity'], size=15, color='cyan')
p.circle('x', 'y', size=10, color='black', source=source)
p.xaxis.major_label_text_font_size = '0pt'

p.line([-1,10],[meaner, meaner], line_width=2, color='red')
p.line([-1,10],[(meaner+stder), (meaner+stder)], line_dash="4 4", line_width=2, color='red')
p.line([-1,10],[(meaner-stder), (meaner-stder)], line_dash="4 4", line_width=2, color='red')


#show(p)

Kanye brags that Jay Z followed his lead by putting Chris Martin on a track (Kingdom Come, Beach Chair). Is this track similarly reviled? 

In [72]:
jz_albums=fetch_albums('JAY Z') 
jz_tracks=fetch_tracks(jz_albums)
jz_explicit=jz_tracks.loc[np.where((jz_tracks['explicit']==True) & (jz_tracks['album'].str.contains('Deluxe')==False) &\
                                   (jz_tracks['album'].str.contains('Acappella')==False))[0]]
jz_explicit.reset_index(drop=True,inplace=True)

Querying Album Tracks
0
20
40
60
80
100
120
140
160
180
200
220
240
260


In [93]:
jz_explicit['count']=pd.Series(range(0, len(jz_explicit)))

source = ColumnDataSource(
        data=dict(
            x=jz_explicit['count'],
            y=jz_explicit['popularity'],
            track=jz_explicit['name'].astype(object),
            album=jz_explicit['album'].astype(str),
        )
    )

TOOLS = [BoxZoomTool(), ResetTool(), HoverTool(tooltips=[("Track","@track"),("album","@album")])]

colormap = {'American Gangster (Explicit Version)': 'red', 'Kingdom Come': 'green',\
            'Magna Carta... Holy Grail (Explicit Version)': 'blue', \
           'The Best Of Both Worlds (Explicit Version)': 'purple', \
            'The Black Album (Explicit)': 'cyan',\
            'Unfinished Business (Explicit)' : 'black', \
            'Watch The Throne (Explicit Version)' : 'orange'}
jz_explicit['color'] = jz_explicit['album'].map(lambda x: colormap[x])


meaner=np.mean(jz_explicit['popularity'])
stder=np.std(jz_explicit['popularity'])

p3 = figure(plot_width=600, plot_height=500, tools=TOOLS)
p3.xaxis.axis_label = "track"
p3.yaxis.axis_label = "Popularity"
p3.circle(jz_explicit.loc[np.where(jz_explicit['name']=='Beach Chair')[0],'count'], jz_explicit.loc[np.where(jz_explicit['name']=='Beach Chair')[0],'popularity'], size=12, color='gold')
p3.circle('x', 'y', size=7,fill_alpha=0.4, color=jz_explicit['color'], source=source)
p3.xaxis.major_label_text_font_size = '0pt'

p3.line([-1,130],[meaner, meaner], line_width=2, color='black')
p3.line([-1,130],[(meaner+stder), (meaner+stder)], line_dash="4 4", line_width=2, color='black')
p3.line([-1,130],[(meaner-stder), (meaner-stder)], line_dash="4 4", line_width=2, color='black')


show(p3)

Nope! 'Beach Chair' is actually a bit more popular than average for Jay Z. And just a note, apparently people don't like it when Hova goes a cappella...

In [94]:
from bokeh.embed import components
script, div = components(p3)

print div

f=open('p7.js', 'w')
f.write(script)
f.close()


<div class="plotdiv" id="dd3c9ae9-4012-4181-8ee6-f93de896a95a"></div>


In [64]:
musician='Kanye West'
artist=sp.search(q='artist:'+musician, type='artist')


In [65]:
artist

{u'artists': {u'href': u'https://api.spotify.com/v1/search?query=artist%3AKanye+West&offset=0&limit=10&type=artist',
  u'items': [{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    u'followers': {u'href': None, u'total': 3375006},
    u'genres': [u'pop rap'],
    u'href': u'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    u'id': u'5K4W6rqBFWDnAN6FQUkS6x',
    u'images': [{u'height': 332,
      u'url': u'https://i.scdn.co/image/fca17e83545e076fafb561569e8d5ec64f87eb8c',
      u'width': 331},
     {u'height': 201,
      u'url': u'https://i.scdn.co/image/01405d36c7091a57b873ee5a91034ba7fc8eab8c',
      u'width': 200},
     {u'height': 64,
      u'url': u'https://i.scdn.co/image/a2fe02386dcb76d7ecda994db995c3fb12694690',
      u'width': 64}],
    u'name': u'Kanye West',
    u'popularity': 92,
    u'type': u'artist',
    u'uri': u'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'},
   {u'external_urls': {u'spotify': u'https://open.spotify.com